In [2]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [4]:
data = Dataset.load_builtin('ml-100k') # 무비렌즈 사이트에서 제공하는 과거 버전의 데이터 세트

trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\정성현/.surprise_data/ml-100k


In [6]:
algo = SVD()
algo.fit(trainset)

In [11]:
predictions = algo.test(testset)
print('prediction type: ', type(predictions), 'size: ',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type:  <class 'list'> size:  25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5558715675878005, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.7315554016591173, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.156820467286122, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.6494797460681787, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.397224965133965, details={'was_impossible': False})]

In [18]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.5558715675878005),
 ('882', '291', 3.7315554016591173),
 ('535', '507', 4.156820467286122)]

In [34]:
uid = str(196)
iid = str(302)
pred = algo.predict(uid,iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.08   {'was_impossible': False}


In [35]:
pred = algo.predict('196', '302')
print(pred)

user: 196        item: 302        r_ui = None   est = 4.08   {'was_impossible': False}


In [37]:
# 테스트 데이터 세트를 이용해 추천 예측 평점과 실제 평점과의 차이 평가
accuracy.rmse(predictions)

RMSE: 0.9492


0.9491827178380478

surprise 패키지는 반드시 데이터 칼럼의 순서가 '사용자아이디','아이템아이디','평점' 순으로 되어 있어야 한다.

In [38]:
import pandas as pd

ratings = pd.read_csv('ratings.csv')

ratings.to_csv('ratings_noh.csv', index=False, header=None)

In [39]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5,5))

data = Dataset.load_from_file('ratings_noh.csv', reader=reader)

In [41]:
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)

algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

#### 판다스 DataFrame에서 Surprise 데이터 세트를 로딩

In [46]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [44]:
reader = Reader(rating_scale=(0.5,5))

data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

In [45]:
algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [47]:
ratings['rating2'] = ratings['rating']

In [48]:
ratings

,userId,movieId,rating,timestamp,rating2
0,1,1,4.0,964982703,4.0
1,1,3,4.0,964981247,4.0
2,1,6,4.0,964982224,4.0
3,1,47,5.0,964983815,5.0
4,1,50,5.0,964982931,5.0
...,...,...,...,...,...
100831,610,166534,4.0,1493848402,4.0
100832,610,168248,5.0,1493850091,5.0
100833,610,168250,5.0,1494273047,5.0
100834,610,168252,5.0,1493846352,5.0


In [49]:
reader = Reader(rating_scale=(0.5,5))

data = Dataset.load_from_df(ratings[['userId','movieId','rating2']], reader)
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

In [50]:
algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516